In [1]:
from llama_stack_client import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
import sys
sys.path.append('.')
from src.client_tools import get_location
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient

# pretty print of the results returned from the model/agent
import sys
sys.path.append('.')
from src.utils import step_printer
from termcolor import cprint

from rich import print

In [2]:
base_url = os.getenv("REMOTE_BASE_URL")

client = LlamaStackClient(
    base_url=base_url
)

# model_id will later be used to pass the name of the desired inference model to Llama Stack Agents/Inference APIs
model_id = "granite-3.2-2b-instruct" # /mnt/models/DeepSeek-R1-0528-Qwen3-8B-Q4_K_M.gguf" # granite-3.2-2b-instruct

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 4096))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Inference Parameters:
        Model: granite-3.2-2b-instruct
        Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 15000}
        stream: True

In [3]:
# Optional: Enter your MCP server URL here
github_mcp_url = os.getenv("REMOTE_GITHUB_MCP_URL") # Optional: enter your MCP server url here

# Get list of registered tools and extract their toolgroup IDs
registered_tools = client.tools.list()
registered_toolgroups = [tool.toolgroup_id for tool in registered_tools]

if "mcp::github" not in registered_toolgroups: # required
    client.toolgroups.register(
        toolgroup_id="mcp::github",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":ocp_github_url},
    )

print(registered_tools)

[
    Tool(
        description='Search the web for information',
        identifier='web_search',
        parameters=[
            Parameter(
                description='The query to search for',
                name='query',
                parameter_type='string',
                required=True,
                default=None
            )
        ],
        provider_id='tavily-search',
        provider_resource_id='web_search',
        tool_host='distribution',
        toolgroup_id='builtin::websearch',
        type='tool',
        metadata=None
    ),
    Tool(
        description='Insert documents into memory',
        identifier='insert_into_memory',
        parameters=[],
        provider_id='rag-runtime',
        provider_resource_id='insert_into_memory',
        tool_host='distribution',
        toolgroup_id='builtin::rag',
        type='tool',
        metadata=None
    ),
    Tool(
        description='Search for information in a database.',
        identifier='knowledge_search',
        parameters=[
            Parameter(
                description='The query to search for. Can be a natural language sentence or keywords.',
                name='query',
                parameter_type='string',
                required=True,
                default=None
            )
        ],
        provider_id='rag-runtime',
        provider_resource_id='knowledge_search',
        tool_host='distribution',
        toolgroup_id='builtin::rag',
        type='tool',
        metadata=None
    ),
    Tool(
        description='Get the current Kubernetes configuration content as a kubeconfig YAML',
        identifier='configuration_view',
        parameters=[
            Parameter(
                description='Return a minified version of the configuration. If set to true, keeps only the 
current-context and the relevant pieces of the configuration for that context. If set to false, all contexts, 
clusters, auth-infos, and users are returned in the configuration. (Optional, default true)',
                name='minified',
                parameter_type='boolean',
                required=True,
                default=None
            )
        ],
        provider_id='model-context-protocol',
        provider_resource_id='configuration_view',
        tool_host='model_context_protocol',
        toolgroup_id='mcp::openshift',
        type='tool',
        metadata={'endpoint': 'http://ocp-mcp-server:8000/sse'}
    ),
    Tool(
        description='List all the Kubernetes events in the current cluster from all namespaces',
        identifier='events_list',
        parameters=[
            Parameter(
                description='Optional Namespace to retrieve the events from. If not provided, will list events from
all namespaces',
                name='namespace',
                parameter_type='string',
                required=True,
                default=None
            )
        ],
        provider_id='model-context-protocol',
        provider_resource_id='events_list',
        tool_host='model_context_protocol',
        toolgroup_id='mcp::openshift',
        type='tool',
        metadata={'endpoint': 'http://ocp-mcp-server:8000/sse'}
    ),
    Tool(
        description='Install a Helm chart in the current or provided namespace',
        identifier='helm_install',
        parameters=[
            Parameter(
                description='Chart reference to install (for example: stable/grafana, 
oci://ghcr.io/nginxinc/charts/nginx-ingress)',
                name='chart',
                parameter_type='string',
                required=True,
                default=None
            ),
            Parameter(
                description='Name of the Helm release (Optional, random name if not provided)',
                name='name',
                parameter_type='string',
                required=True,
                default=None
            ),
            Parameter(
                description='Namespace to install the Helm chart in (Opti

In [4]:
agent = ReActAgent(
            client=client,
            model=model_id,
            tools=["mcp::github"],
            response_format={
                "type": "json_schema",
                "json_schema": ReActOutput.model_json_schema(),
            },
            sampling_params=sampling_params,
        )

user_prompts = ["""Get file content using GitHub MCP owner=eformat, repo=welcome, path=README.md, ref=master
                Use the following principles to generate the code summary from the provided file content
                - Concisely summarize what the code is doing.
                - Include small code snippets to create a comprehensive summary
                Return the generated summary as follows:
                [Start Summary]
                <insert summary here>
                [End Summary]
                """
                ]

session_id = agent.create_session("web-session")
for prompt in user_prompts:
    print("\n"+"="*50)
    cprint(f"Processing user query: {prompt}", "blue")
    print("="*50)
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
        stream=stream
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 

==================================================

Processing user query: Get file content using GitHub MCP owner=eformat, repo=welcome, path=README.md, ref=master
                Use the following principles to generate the code summary from the provided file content
                - Concisely summarize what the code is doing.
                - Include small code snippets to create a comprehensive summary
                Return the generated summary as follows:
                [Start Summary]
                <insert summary here>
                [End Summary]
                


==================================================

inference> {
  "thought": "To get the file content, I'll use the 'get_file_contents' tool. After obtaining the content, I'll summarize the purpose of the README.md file, including any relevant code snippets, and present it in the requested format.",
  "action": {
    "tool_name": "get_file_contents",
    "tool_params": [
      {
        "name": "owner",
        "value": "eformat"
      },
      {
        "name": "repo",
        "value": "welcome"
      },
      {
        "name": "path",
        "value": "README.md"
      },
      {
        "name": "branch",
        "value": "master"
      }
    ]
  },
  "answer": null
}
tool_execution> Tool:get_file_contents Response:{"type":"text","text":"{\n  \"name\": \"README.md\",\n  \"path\": \"README.md\",\n  \"sha\": \"1a5ef69e3e418465836839757c727899622802d0\",\n  \"size\": 740,\n  \"url\": \"https://api.github.com/repos/eformat/welcome/contents/README.md?ref=master\",\n  \"html_url\": \"https://github.com/eformat/welcome/blob/master/README.md